In [1]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from config import api_key
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy
%matplotlib inline

In [2]:
quandl.ApiConfig.api_key = api_key
#Variance Risk Premium: https://sites.google.com/site/haozhouspersonalhomepage
#EOM Nominal S&P, Nominal Dividends / Nominal Risk Free Rate: http://www.hec.unil.ch/agoyal/
#Consumer Sentiment: http://www.sca.isr.umich.edu/

In [3]:
ticker = 'SPY'
transform = 'normalize'
start_date = '1989-01-01'
end_date = '2018-12-31'
period = 'monthly'

In [4]:
#Access data from Quandl
ticker_data = quandl.get('EOD/' + ticker, start_date=start_date, end_date=end_date, collapse=period)
risk_free_rate = quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date, collapse=period)
fed_funds_rate = quandl.get('FRED/FEDFUNDS', start_date=start_date, end_date=end_date, collapse=period)

#Confirm results are same size:
if ticker_data.shape[0] == risk_free_rate.shape[0] == fed_funds_rate.shape[0]:
    
    print(f"All dataframes are {ticker_data.shape[0]} rows")

else:
    print('Dataframes are not the same size:',  
          f'ticker_data is {ticker_data.shape[0]}, ',
          f'risk_free_rate is {risk_free_rate.shape[0]}, ',
          f'fed_funds_rate is {fed_funds_rate.shape[0]}.');

Dataframes are not the same size: ticker_data is 312,  risk_free_rate is 360,  fed_funds_rate is 360.


In [5]:
#Make Function to Create Return Column for Risk Free Rate & Fed Funds Rate
def make_return_column(data):
    
    list_return = []

    for i in range(len(data)):
        x = (data.iloc[i]['Value']/12)/100
        list_return.append(x)
        
    temp_df = pd.DataFrame(list_return)
    
    data = pd.concat([data.reset_index(), temp_df.reset_index(drop=True)], axis=1)
    data = data.rename(columns={0: 'return'})
    
    return data

risk_free_df = make_return_column(risk_free_rate)

In [6]:
fed_funds_df = pd.DataFrame(fed_funds_rate.reset_index())

In [7]:
fed_funds_df18 = fed_funds_df[fed_funds_df['Date'] >= '2018-01-01']
fed_funds_df = fed_funds_df[fed_funds_df['Date'] < '2018-01-01']

In [8]:
fed_funds_df18 = fed_funds_df18['Value']
fed_funds_df = fed_funds_df['Value']

In [10]:
"""
BUILD DATASET

Monthly >>>
Index: S&P 500 Index Data
D12: Dividends are 12-month moving sums of dividends paid on the S&P 500 index (Goyal, Welch) 
     -- Confirmed with Y Charts
Rfree: Nominal Risk Free Rate
"""
goyal = pd.read_excel('PredictorData2017.xlsx')
df = pd.DataFrame(goyal)

#New DF with cleaned up Columns
df_cols = ['yyyymm', 'Index', 'D12', 'Rfree']
df = df[df_cols]

#Convert yyyymm to Datetime then to Period
yyyymm = pd.to_datetime(df['yyyymm'], format='%Y%m').dt.strftime('%Y-%m')
df['yyyymm'] = pd.to_datetime(yyyymm, format='%Y-%m')

#Drop rows prior to 1990-01
df = df[df['yyyymm'] >= '1990-01'];

In [12]:
"""
VRP: Variance Risk Premium = IV - RV
IV: Implied Variance = VIX-Squared/12
RV: Realized Variance = Sum of Squared 5-minute log returns of the S&P 500 Index
"""
vrp = pd.read_fwf('VRPtable.txt')
df_vrp = pd.DataFrame(vrp)

#Drop 2018 Rows & create new VRP 2018 DF
df_vrp2018 = df_vrp[df_vrp['Year'] >= 2018]
df_vrp = df_vrp[df_vrp['Year'] < 2018]


#Drop Unnecessary Columns
df_vrp2018 = df_vrp2018[['VRP']]
df_vrp = df_vrp[['VRP']];

In [13]:
#Concat Index DF with VRP DF
df = pd.concat([df.reset_index(drop=True), df_vrp.reset_index(drop=True)], axis=1)

In [14]:
#Rename yyyymm Column to EOM
df = df.rename(columns={'yyyymm': 'EOM'})

In [15]:
#New DF from Index / Div / Rfree Data -> Need to Add 1989 Data.
df_1989 = pd.DataFrame(goyal)
df_1989_cols = ['yyyymm', 'Index', 'D12', 'Rfree']

df_1989 = df_1989[df_1989_cols]
yyyymm_1989 = pd.to_datetime(df_1989['yyyymm'], format='%Y%m').dt.strftime('%Y-%m')
df_1989['yyyymm']  = pd.to_datetime(yyyymm_1989, format='%Y-%m')

df_1989 = df_1989[(df_1989['yyyymm'] >= '1989-01') & (df_1989['yyyymm'] < '1990-01')]

df_1989 = df_1989.rename(columns={'yyyymm': 'EOM'})

df = pd.concat([df_1989, df], axis=0, ignore_index=True, sort=False)

In [16]:
#Concat Fed Funds Rate
df = pd.concat([df.reset_index(drop=True), fed_funds_df.reset_index(drop=True)], axis=1)
df = df.rename(columns={'Value': 'fed_funds'})

In [18]:
sentiment_file = 'tbmics.csv'
sentiment = pd.read_csv(sentiment_file)

sentiment18 = sentiment[(sentiment['YYYY'] >= 2018) & (sentiment['YYYY'] < 2019)]
sentiment = sentiment[(sentiment['YYYY'] >= 1989) & (sentiment['YYYY'] < 2018)]


In [19]:
sentiment18 = sentiment18['ICS_ALL']
sentiment = sentiment['ICS_ALL']

In [20]:
df = pd.concat([df.reset_index(drop=True), sentiment.reset_index(drop=True)], axis=1)

In [24]:
"""
2018 Data
"""

#2018 S&P 500 Dividends
dividends18 = 'Dividend_History.csv'
div_data18 = pd.read_csv(dividends18)
div18 = pd.DataFrame(div_data18)

div18 = div18[['Value']]
div18 = div18.rename(columns={'Value': 'D12'})

#2018 S&P Index Data from Yahoo
f2018 = '^GSPC.csv'
data = pd.read_csv(f2018)
sp18 = pd.DataFrame(data)
sp18 = sp18[['Date', 'Close']]

sp18 = pd.concat([sp18.reset_index(drop=True), df_vrp2018.reset_index(drop=True)], axis=1);

In [25]:
#Concat 2018 DF with Risk Free Rate Data
rfdf18 = risk_free_df['return'].tail(12)
sp18 = pd.concat([sp18.reset_index(drop=True), rfdf18.reset_index(drop=True)], axis=1)
sp18 = sp18.rename(columns={'return': 'Rfree'})

In [26]:
#Concat 2018 federal funds rate return
sp18 = pd.concat([sp18.reset_index(drop=True), fed_funds_df18.reset_index(drop=True)], axis=1)
sp18 = sp18.rename(columns={'Value': 'fed_funds'})

In [27]:
#Cocnat 2018 Dividends and clean up columns
sp18 = pd.concat([sp18.reset_index(drop=True), div18.reset_index(drop=True)], axis=1)
sp18 = sp18.rename(columns={'Close': 'Index', 'Date': 'EOM'})
sp18['EOM'] = pd.to_datetime(sp18['EOM'])

In [28]:
#Concat 2018 Sentiment
sp18 = pd.concat([sp18.reset_index(drop=True), sentiment18.reset_index(drop=True)], axis=1)

In [29]:
#Reorder Columns
cols = ['EOM', 'Index', 'D12', 'Rfree', 'fed_funds', 'VRP', 'ICS_ALL']
sp18 = sp18[cols]
df = df[cols]

In [30]:
#Create Dataframe with all Data from 1989 to 2018
df_all = pd.concat([df.reset_index(drop=True), sp18.reset_index(drop=True)], axis=0)

In [31]:
def build_df(df):
    
#Number of Periods in Variance Risk Premium
    vrp_rolling_periods = 3

    #Create Column with PD Ratio
    df['pd'] = (df['Index'] / df['D12'])

    #Create Column with Fed Funds Rate Delta (12 Month)
    df['ff12_D'] = df['fed_funds'] - df['fed_funds'].shift(12)

    #Create Column with Rolling Variance Risk Premium Rolling
    df['vrp3'] = df['VRP'].rolling(vrp_rolling_periods).mean()

    #Create Column with Sentiment Delta
    df['sent12_D'] = (df['ICS_ALL'] - df['ICS_ALL'].shift(12))
    
    #Create Column with Excess Return on Index Relative to the RFR in percent
    df['ersf'] = (((df['Index'] + df['D12']/12 - df['Index'].shift(1)) / df['Index'].shift(1)) - df['Rfree']) * 100

#Create Column with Excess Return on Index Relative to the RFR in percent
    df['ersf_t1'] = (((df['Index'].shift(-1) + df['D12'].shift(-1)/12 - df['Index']) / df['Index']) - df['Rfree'].shift(-1)) * 100

#Create Column with 1-month Delta in ersf
    df['ersf_D'] = df['ersf'] - df['ersf'].shift(1)

#Create column with Sentiment X ersf Delta
    df['sent_x_ersf_D'] = df['sent12_D'] * df['ersf_D']
    
    return df

df_all = build_df(df_all)

In [32]:
# Create Filter for 2003 to 2018
filt_2003 = (df_all['EOM'] >= '2003-01-01')
df0318 = df_all[filt_2003]

#Cut off 2013
df0418 = df0318[12:]

In [34]:
google_trends_file = 'GoogleTrends_2004-2018.csv'

google_data = pd.read_csv(google_trends_file)
google = pd.DataFrame(google_data)
google = google[1:]

google = google.rename(columns={'Category: All categories': 'Google'})
google['Google'] = google['Google'].astype('float64')
google['Google_D'] = google['Google'] - google['Google'].shift(1)
google.fillna(google.mean(), inplace=True)

df0418 = pd.concat([df0418.reset_index(drop=True), google.reset_index(drop=True)], axis=1)

df0418.head()

,EOM,Index,D12,Rfree,fed_funds,VRP,ICS_ALL,pd,ff12_D,vrp3,sent12_D,ersf,ersf_t1,ersf_D,sent_x_ersf_D,Google,Google_D
0,2004-01-01,1131.13,17.598000,0.000733,1.00,14.0860,103.8,64.276054,-0.24,16.151067,21.4,1.786198,1.274621,-3.352254,-71.738239,28.0,0.273743
1,2004-02-01,1144.94,17.811000,0.000775,1.01,9.8728,94.4,64.282747,-0.25,14.917533,14.5,1.274621,-1.583041,-0.511577,-7.417861,28.0,0.000000
2,2004-03-01,1126.21,18.024000,0.000783,1.00,9.4462,95.8,62.483910,-0.25,11.135000,18.2,-1.583041,-1.622623,-2.857662,-52.009454,26.0,-2.000000
3,2004-04-01,1107.30,18.216667,0.000783,1.00,13.4427,94.2,60.784995,-0.26,10.920567,8.2,-1.622623,1.261890,-0.039582,-0.324572,25.0,-1.000000
4,2004-05-01,1120.68,18.409333,0.000850,1.00,5.2403,90.2,60.875643,-0.26,9.376400,-1.9,1.261890,1.831398,2.884513,-5.480574,23.0,-2.000000


In [35]:
export_csv = df_all.to_csv (r'/Users/maks_p/Flatiron/PROJECT/Momentum_X_Sentiment/1989_2018_Data.csv', index = None)

In [36]:
export_csv = df0418.to_csv (r'/Users/maks_p/Flatiron/PROJECT/Momentum_X_Sentiment/2004_to_2018_withGoogle.csv', index = None)